In [ ]:
import pandas as pd
import multiprocessing
import time
import numpy as np
import csv
import sys
from itertools import chain

# Aumentar el límite del campo CSV, esencial para archivos grandes
csv.field_size_limit(sys.maxsize)
# Solución a la advertencia de Pandas sobre copias encadenadas
pd.options.mode.chained_assignment = None

# --- ALGORITMOS DE ORDENAMIENTO Y DEPURACIÓN ---

def merge_sort(lista):
    """Implementa el algoritmo Merge Sort (Ordenamiento por mezcla)."""
    if len(lista) <= 1:
        return lista
    
    mitad = len(lista) // 2
    lista_izquierda = merge_sort(lista[:mitad])
    lista_derecha = merge_sort(lista[mitad:])
    
    return unir_listas(lista_izquierda, lista_derecha)

def unir_listas(lista1, lista2):
    """Función auxiliar para Merge Sort que une dos listas ordenadas."""
    resultado = []
    i = j = 0
    while i < len(lista1) and j < len(lista2):
        # Criterio de ordenamiento: de mayor a menor temperatura
        if lista1[i][2] >= lista2[j][2]:
            resultado.append(lista1[i])
            i += 1
        else:
            resultado.append(lista2[j])
            j += 1
    resultado.extend(lista1[i:])
    resultado.extend(lista2[j:])
    return resultado

def quick_sort_depuracion(lista):
    """
    Usa el estilo de partición de Quick Sort para depurar/filtrar la lista.
    Filtra valores que no son números válidos o son 0.0.
    """
    n = len(lista)
    i = -1  # Índice del último elemento válido encontrado
    for j in range(n):
        valor_temp = lista[j][2]
        # Si es numérico y diferente de 0.0, es válido
        if isinstance(valor_temp, (int, float)) and valor_temp != 0.0:
            i += 1
            lista[i], lista[j] = lista[j], lista[i] # Swap en Python
    return i + 1

# --- IMPLEMENTACIÓN DE LA FÓRMULA DE PROMEDIO ANUAL ---

def calcular_promedios(lista_datos):
    """
    Calcula la temperatura promedio anual por ciudad (suma/n) usando la lógica de diccionarios.
    """
    # Diccionario para guardar la suma de las temperaturas (ciudad|anio)
    sumas = {}
    # Diccionario para contar los días (n) por ciudad|anio
    conteo = {}

    for ciudad, anio, temp in lista_datos:
        clave = f"{ciudad}|{anio}"

        # Lógica para sumar las temperaturas
        sumas[clave] = sumas.get(clave, 0.0) + temp
        
        # Lógica para contar los días
        conteo[clave] = conteo.get(clave, 0) + 1

    lista_promedios = []

    # Aplicar la fórmula: Promedio = Suma / N
    for clave in sumas.keys():
        suma_total = sumas[clave]
        num_dias = conteo[clave]

        promedio = suma_total / num_dias

        # Descomponer la clave
        ciudad, anio = clave.split("|")

        # Agregar el registro de promedio anual (Ciudad, Año, Promedio)
        lista_promedios.append((ciudad, anio, promedio))

    return lista_promedios

# --- LÓGICA DE PROCESAMIENTO (CON MULTIPROCESAMIENTO) ---

def procesar_bloque(bloque):
    """
    Realiza la limpieza a nivel de fila de forma paralela y devuelve filas limpias.
    """
    if bloque.empty:
        return pd.DataFrame()

    try:
        bloque = bloque.copy()

        # 1. Limpieza y conversión a numérico
        bloque['AvgTemperature'] = bloque['AvgTemperature'].replace(r'["\(\)-]', '', regex=True)
        bloque['AvgTemperature'] = pd.to_numeric(bloque['AvgTemperature'], errors='coerce')

        # 2. Limpieza de fecha y extracción del año
        bloque['Fecha'] = bloque['Fecha'].replace(r'["\(\)]', '', regex=True)
        bloque = bloque.dropna(subset=['AvgTemperature', 'Fecha'])
        
        bloque['Anio'] = bloque['Fecha'].str.split('/').str[2]
        bloque = bloque.dropna(subset=['Anio'])

        # 3. Filtrado extremo
        bloque = bloque[bloque['AvgTemperature'] > -90]

        # 4. Devolver las columnas necesarias para el cálculo del promedio:
        return bloque[['City', 'Anio', 'AvgTemperature']]

    except Exception as e:
        return pd.DataFrame()


def procesar_en_paralelo(bloques_a_procesar, funcion):
    """
    Divide y mapea la función 'funcion' a los bloques en paralelo.
    """
    num_nucleos = multiprocessing.cpu_count()
    print(f"\n[ PROCESAMIENTO PARALELO] Usando {num_nucleos} núcleos para {len(bloques_a_procesar)} bloques.")

    with multiprocessing.Pool(num_nucleos) as pool:
        resultados = pool.map(funcion, bloques_a_procesar)

    return resultados

# --- FUNCIÓN PRINCIPAL (MAIN) ---

def principal():
    tiempo_INICIAL = time.time()
    archivo_entrada = 'temperaturas.csv'
    archivo_salida = 'promedios_ordenados.csv'
    TAMANO_BLOQUE = 100000

    print("\n=======================================================")
    print(" INICIANDO PROCESADOR DE DATOS DE TEMPERATURA")
    print("=======================================================")

    # 1. LECTURA POR BLOQUES Y LIMPIEZA PARALELA
    print("\n[PASO 1]  Leyendo y Limpiando filas crudas en paralelo...")
    inicio_lectura = time.time()
    bloques = []
    try:
        iterador_csv = pd.read_csv(
            archivo_entrada,
            usecols=['City', 'AvgTemperature', 'Fecha'],
            chunksize=TAMANO_BLOQUE,
            encoding='utf-8',
            low_memory=True,
            skip_blank_lines=True
        )
        for bloque in iterador_csv:
            bloques.append(bloque)
    except Exception as e:
        print(f"\n ERROR: Al leer el archivo {archivo_entrada}. Detalles: {e}")
        return
    
    resultados = procesar_en_paralelo(bloques, procesar_bloque)
    resultados_validos = [res for res in resultados if not res.empty]
    if not resultados_validos:
        print("\n Advertencia: No se obtuvieron datos válidos de los procesos paralelos.")
        return

    # Consolidar todos los DataFrames de filas limpias
    df_limpio_final = pd.concat(resultados_validos, ignore_index=True)
    
    # Convertir el DataFrame a lista de tuplas
    lista_datos_limpia = df_limpio_final.to_records(index=False).tolist()
    fin_lectura = time.time()
    
    print("-------------------------------------------------------")
    print(f"   Tiempo de lectura y limpieza paralela: {fin_lectura - inicio_lectura:.4f} segundos")
    print(f"   Filas crudas válidas consolidadas: {len(lista_datos_limpia):,} filas.")
    print("-------------------------------------------------------")


    # 2. CÁLCULO DE PROMEDIOS ANUALES se usa la formula)
    print("\n[PASO 2]  Calculando Promedios Anuales (Suma/N)...")
    inicio_promedios = time.time()
    
    # El resultado es la lista de promedios anuales 
    lista_promedios_calculados = calcular_promedios(lista_datos_limpia)
    
    fin_promedios = time.time()
    
    print(f"   Tiempo de cálculo de promedios: {fin_promedios - inicio_promedios:.4f} segundos")
    print(f"   Registros de promedios anuales: {len(lista_promedios_calculados):,} registros.")
    print("-------------------------------------------------------")


    # 3. DEPURACIÓN NIVEL 2 (QuickSort)
    print("\n[PASO 3]  Depuración final (QuickSort) de los promedios...")
    inicio_depuracion = time.time()
    num_validos = quick_sort_depuracion(lista_promedios_calculados)
    lista_depurada = lista_promedios_calculados[:num_validos]
    fin_depuracion = time.time()
    print(f"   Tiempo de depuración: {fin_depuracion - inicio_depuracion:.4f} segundos")
    print(f"   Promedios finales a ordenar: {num_validos:,} registros.")
    print("-------------------------------------------------------")


    # 4. ORDENAMIENTO (Merge Sort)
    print("\n[PASO 4]  Ordenamiento (Merge Sort) de los promedios...")
    inicio_ordenamiento = time.time()
    datos_ordenados = merge_sort(lista_depurada)
    fin_ordenamiento = time.time()
    print(f"   Tiempo de ordenamiento: {fin_ordenamiento - inicio_ordenamiento:.4f} segundos")
    print("-------------------------------------------------------")


    # 5. ESCRITURA DE REPORTE
    print(f"\n[PASO 5]  Generando archivo CSV: {archivo_salida}")
    inicio_escritura = time.time()
    try:
        # Se usa codificación UTF-8 para evitar errores con caracteres especiales
        with open(archivo_salida, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['Anio', 'Ciudad', 'TemperaturaPromedio'])
            
            for row in datos_ordenados:
                ciudad, anio, temp = row
                writer.writerow([anio, f'"{ciudad}"', f'{temp:.2f}'])

    except Exception as e:
        print(f"\n ERROR: Al escribir el archivo. Detalles: {e}")
        return
    fin_escritura = time.time()
    print(f"    Tiempo de escritura: {fin_escritura - inicio_escritura:.4f} segundos")


    tiempo_FINAL = time.time()
    print("\n=======================================================")
    print(f" PROCESO COMPLETADO. Tiempo total: {tiempo_FINAL - tiempo_INICIAL:.4f} segundos.")
    print("=======================================================")


if __name__ == '__main__':
    # Necesario para que el multiprocesamiento funcione correctamente
    multiprocessing.freeze_support()
    principal()


 INICIANDO PROCESADOR DE DATOS DE TEMPERATURA

[PASO 1]  Leyendo y Limpiando filas crudas en paralelo...

[ PROCESAMIENTO PARALELO] Usando 8 núcleos para 11 bloques.
-------------------------------------------------------
   Tiempo de lectura y limpieza paralela: 3.0196 segundos
   Filas crudas válidas consolidadas: 992,090 filas.
-------------------------------------------------------

[PASO 2]  Calculando Promedios Anuales (Suma/N)...
   Tiempo de cálculo de promedios: 0.2385 segundos
   Registros de promedios anuales: 2,912 registros.
-------------------------------------------------------

[PASO 3]  Depuración final (QuickSort) de los promedios...
   Tiempo de depuración: 0.0004 segundos
   Promedios finales a ordenar: 2,912 registros.
-------------------------------------------------------

[PASO 4]  Ordenamiento (Merge Sort) de los promedios...
   Tiempo de ordenamiento: 0.0064 segundos
-------------------------------------------------------

[PASO 5]  Generando archivo CSV: pro